# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env") 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [5]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPStatusError as e:
            # Handle 302 redirect manually if follow_redirects fails
            if e.response.status_code == 302 and "location" in e.response.headers:
                redirect_url = e.response.headers["location"]
                if not redirect_url.startswith("http"):
                    # handle relative redirects
                    redirect_url = urljoin(url, redirect_url)
                try:
                    response = await client.get(redirect_url, headers=headers)
                    response.raise_for_status()
                except Exception as e2:
                    print(f"Redirect request failed: {e2}")
                    return ""
            else:
                print(f"Request failed: {e}")
                return ""
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [6]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 10

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]
    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Google Search Results: 10


삼성전자 인덕션 중 2구가 아닌 3구 이상 또는 다양한 화구 수를 가진 모델 중 추천할 만한 제품과 그 주요 기능 및 장점은 아래와 같습니다. 최신 2025년 5월 기준 정보를 바탕으로 상세히 안내해 드립니다.

---

## 1. 삼성전자 비스포크 AI 3구 인덕션 NZ63DB607CF
- **출력**  
  - 대화구: 3,400W  
  - 중화구: 2,600W  
  - 소화구: 1,200W  
- **주요 기능 및 장점**  
  - 최상급 화력으로 빠른 조리 속도 제공  
  - AI 넘침 방지 모드 탑재로 조리 중 끓음 걱정 감소  
  - 저소음 DCN 기능으로 쾌적한 사용 환경 조성  
  - 대화형 알림창과 심플해진 1개 조작부, 프레임리스 디자인으로 세련된 외관  
  - 두께가 얇아 식기세척기와 함께 설치해도 튀어나옴 최소화  
  - 가격은 100만 원 미만대로 가성비 개선  
  - 단, 상판은 SCHOTT가 아닌 NEG 제품이며, 듀얼링 대화구는 미적용  

---

## 2. 삼성전자 비스포크 AI 3구 인덕션 NZ63DB607CABS
- **출력**  
  - NZ63DB607CF와 동급  
- **주요 기능 및 장점**  
  - 상판에 업계 최고 브랜드 SCHOTT 적용  
  - 스마트컨트롤로 외부에서 전원 제어 가능  
  - 비스포크 AI 모델 중 상위급 성능과 기능 탑재  
  - 가격은 NZ63DB607CF보다 다소 높으나 성능 대비 합리적 선택 가능  

---

## 3. 삼성전자 비스포크 3구 인덕션 NZ63D650BXE
- **출력**  
  - 대화구 최대 3,400W (동급)  
- **주요 기능 및 장점**  
  - AI 끓음 감지 기능 미탑재 (기능은 다소 하향)  
  - 조작부가 각 화구별로 3개 탑재  
  - 두께가 비교적 두껍지만, 가격이 66~79만원대로 저렴한 편  
  - 예산을 절약하고자 하는 소비자에게 적합  

---

## 4. 삼성전자 비스포크 프리미엄 3구 인덕션 NZ63DB657CA*S
- **출력**  
  - 각 화구 일반 출력 2,000W 이상 (프리미엄급)  
- **주요 기능 및 장점**  
  - 4분할 플렉스 존으로 다양한 크기 용기 자유롭게 사용 가능  
  - LCD 대화형 알림창, AI 끓음 감지, DCN 저소음 기능 탑재  
  - 앱 연동을 통한 원격 제어 가능  
  - 두께가 얇아 설치 시 공간 활용 우수  
  - 가격은 프리미엄임에도 경쟁사 일반 3구 인덕션 수준으로 합리적  

---

## 5. 삼성전자 하이브리드 인덕션 NZ63T4507AK (인덕션+하이라이트 혼합)
- **출력**  
  - 인덕션 대화구: 3,300W  
  - 인덕션 중화구: 1,300W  
  - 하이라이트 1구: 1,050W  
- **주요 기능 및 장점**  
  - 인덕션 화력과 하이라이트의 용기 제한 없는 편리함을 동시에 제공  
  - SCHOTT사의 고품질 상판 사용  
  - 합리적인 가격에 하이브리드 조합 선호하는 사용자 추천  
  - 메인 대화구 인덕션으로 강력한 화력 제공  

---

# 요약 및 추천 가이드

| 모델명                 | 화구수 | 대화구 출력 | 주요 특징                                     | 가격대(원)          |
|----------------------|--------|-------------|--------------------------------------------|-------------------|
| NZ63DB607CF          | 3구    | 3,400W      | AI 넘침 방지, 저소음, 프레임리스, 가성비 최고 | 약 80~100만        |
| NZ63DB607CABS        | 3구    | 3,400W      | SCHOTT 상판, 스마트컨트롤, 고급형              | 약 96~115만        |
| NZ63D650BXE          | 3구    | 3,400W      | 기본형, AI 기능 미탑재, 저렴한 가격             | 약 66~79만         |
| NZ63DB657CA*S        | 3구    | 2,000W 이상 | 4분할 플렉스 존, AI 끓음 감지, 앱 연동            | 약 90~110만        |
| NZ63T4507AK (하이브리드) | 3구    | 3,300W (인덕션) | 인덕션+하이라이트 조합, SCHOTT 상판, 편리한 용기 사용 | 약 110~130만       |

---

필요하신 경우 각 제품별 추가 편의 기능이나 디자인, 설치 조건 등 상세 안내도 도와드리겠습니다.  
삼성전자 3구 이상 인덕션은 뛰어난 출력과 첨단 AI 편의 기능, 프리미엄 상판 소재를 적용해 빠르고 안전한 조리 환경을 제공합니다. 가성비와 고급 기능을 동시에 원하시면 비스포크 AI 모델군을, 하이브리드 구성을 원하시면 NZ63T4507AK 모델을 권장합니다.

궁금하신 점이나 다른 제품 추천 원하시면 언제든 문의해 주세요!

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV 모델을 추천해 주세요. 크기, 기능, 가격대 등 다양한 옵션과 장점을 포함해서 알려 주세요.
Google Search Results: 10
Request failed: 
Request failed: 


부모님께 선물하기 좋은 삼성전자 TV 모델을 추천드리겠습니다. 2025년 5월 기준 최신 정보와 사용자 경험을 토대로 다음과 같이 정리했습니다.

---

## 1. 추천 모델 및 크기

| 모델명                    | 화면 크기       | 패널 종류         | 예상 가격대 (원)   | 특징 및 장점                                                                                     |
|---------------------------|-----------------|-------------------|-------------------|-------------------------------------------------------------------------------------------------|
| 삼성 QLED Q67C 시리즈     | 65~75인치       | QLED              | 100~130만원대     | 뛰어난 화질과 명암비, 선명한 색감, HDR 지원으로 자연 다큐, 여행 영상 감상에 적합                  |
| 삼성 Neo QLED N60C 시리즈 | 65~75인치       | Mini LED QLED     | 120~160만원대     | 고급형 미니 LED 탑재로 더 선명하고 깊은 명암비 제공, AI 화질 개선 기능 탑재                       |
| 삼성 Crystal UHD AU8000 시리즈 | 65~75인치  | LED (Crystal UHD) | 70~100만원대      | 가성비 좋은 모델, 기본적인 화질과 스마트 기능 제공, 대형 TV 필요하지만 예산이 제한적일 때 적합     |

---

## 2. 크기 선택 가이드

- **75인치**: 거실이 넓고 큰 화면을 선호하실 경우 추천. 영상 몰입감이 뛰어남.
- **65인치**: 일반적인 거실 크기에 적합하며, 가격 부담이 덜함.
- **주요 참고**: 부모님 세대는 큰 사이즈를 선호하지만, 실제 시청 거리와 공간을 고려해 결정하는 것이 중요합니다.

---

## 3. 기능 및 장점

- **QLED 기술**: 일반 LED 대비 더 풍부하고 정확한 색감을 제공하여 자연 다큐, 여행기 영상 시청에 적합합니다.
- **AI 업스케일링**: 낮은 해상도 영상도 고화질로 변환해 주어 TV 방송, 넷플릭스 등 다양한 콘텐츠를 쾌적하게 감상할 수 있습니다.
- **스마트 TV 기능**: 삼성 스마트허브를 통한 간편한 앱 접속, 음성 인식 리모컨 지원으로 사용 편리성 높음.
- **내구성 및 AS**: 삼성전자는 국내 대기업 브랜드로서 A/S 서비스가 뛰어나 부모님께 안심하고 선물하기 좋습니다.

---

## 4. 가격대 및 구매 팁

- **예산 130만원 내외**라면 QLED 75인치 중 Q67C 시리즈가 가성비 좋고 화질 우수해 추천드립니다.
- **예산 조금 낮추고 싶다면** Crystal UHD 75인치 모델도 무난하며, 기본적인 TV 사용에 충실합니다.
- **할인 시즌 활용**: 연말, 명절, 대형 세일 기간을 노리면 10~20% 할인도 가능하니 구매 전 가격 추적을 권장합니다.
- **구매처**: 삼성 공식 온라인몰, SSG.COM, 쿠팡 등 대형 이커머스에서 적립 혜택과 사은품도 확인하세요.

---

## 5. 사용자 후기 참고

- 부모님 세대는 브랜드 신뢰도가 높아 삼성, LG 제품 선호도가 매우 높습니다.
- TCL 등 저가 브랜드도 있으나, 초기 세팅과 A/S 불편함으로 인해 부모님 TV 선물용으로는 삼성 제품 추천이 많습니다.
- QLED 60~70 시리즈는 화질, 성능 면에서 보급형 대비 확실한 업그레이드를 경험할 수 있습니다.

---

# 요약

| 추천 모델        | 크기      | 가격대          | 추천 이유                                    |
|-----------------|-----------|----------------|--------------------------------------------|
| 삼성 QLED Q67C   | 75인치    | 약 120~130만원 | 화질 우수, A/S 좋음, 부모님 선물로 인기       |
| 삼성 Neo QLED N60C | 65~75인치 | 약 130~160만원 | 고급 화질과 기능, 조금 더 투자 가능 시 추천    |
| 삼성 Crystal UHD AU8000 | 75인치 | 약 70~100만원  | 가성비 좋은 기본형, 예산 제한 시 적합          |

---

부모님께서 편안하게 TV를 즐기시고 브랜드 신뢰도와 A/S 걱정 없는 삼성전자의 최신 QLED 또는 Neo QLED 모델을 추천드립니다. 필요하시면 구체적인 모델별 상세 스펙과 구매처 안내도 도와드리겠습니다!

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 개선 사항 2024년 대비
LLM Query: 삼성전자 2025년 신제품이 2024년 제품과 비교해서 어떤 점이 좋아졌는지, 주요 개선 사항과 업그레이드된 기능을 알려주세요.
Google Search Results: 10
Request failed: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)


...

안녕하세요! 삼성전자에서 판매하는 JBL과 하만카돈(Harman Kardon) 브랜드의 스피커 및 오디오 제품에 대해 2025년 5월 기준으로 주요 차이점과 특징을 음질, 디자인, 가격대, 기술적 측면에서 자세히 설명해 드리겠습니다.

---

# 삼성전자 판매 JBL vs 하만카돈 스피커/오디오 주요 차이점 및 특징

| 구분         | JBL                                              | 하만카돈 (Harman Kardon)                          |
|--------------|-------------------------------------------------|-------------------------------------------------|
| **브랜드 특징**  | 스포츠 및 게이밍 오디오에 강점, 대중적이고 실용적 | 프리미엄 오디오 브랜드, 고급스러운 음향과 디자인 |
| **음질 특성**   | 강력한 베이스와 선명한 중고음, 몰입감 있는 사운드 제공<br>예: JBL Quantum 시리즈는 공간 음향 기술과 헤드 트래킹 지원으로 게이밍에서 뛰어난 위치감 제공 | 균형 잡힌 사운드, 부드러운 고음과 풍부한 중저음, 하이파이(Hi-Fi) 음질에 초점<br>고해상도 오디오 재생에 최적화 |
| **디자인**    | 젊고 역동적인 디자인, RGB 조명 등 게이밍 및 실내 인테리어 친화적 요소 포함<br>내구성과 휴대성 강조 | 세련되고 미니멀한 고급스러운 디자인, 메탈 및 고급 소재 사용<br>가정용 프리미엄 인테리어 제품에 적합 |
| **가격대**    | 중저가부터 중가형 제품군 중심<br>예: JBL Quantum 시리즈 헤드셋 10~30만원대, 스피커도 10~50만원대 다양 | 중고가~고가형 프리미엄 라인업 중심<br>하만카돈 스피커 및 오디오 시스템은 30만원 이상부터 고급형은 수백만원대도 있음 |
| **주요 기술 및 기능** | - JBL QuantumSURROUND, QuantumSPHERE 360(헤드 트래킹 기반 360도 공간 음향)<br>- 커스터마이징 가능한 EQ, RGB LED 조명<br>- 게임 최적화 오디오 소프트웨어 제공<br>- 휴대용, 무선 연결 지원이 강점 | - 하이레졸루션 오디오 지원<br>- 프리미엄 DSP(디지털 신호 처리) 기술 탑재<br>- 고품질 소재 및 정밀 튜닝 통한 고음질 실현<br>- 일부 모델은 무선 멀티룸, 스마트 홈 연동 지원 |

---

## 상세 설명

### 1. 음질 비교
- **JBL**은 게이밍과 스포츠 음향에 특화되어 강력한 베이스와 정확한 중고음, 몰입감 있는 공간 음향 기술(QuantumSURROUND, QuantumSPHERE 360)으로 게이머들에게 최적화된 청취 경험을 제공합니다. 주변 환경의 미세한 소리까지 정밀하게 들리도록 설계되어 경쟁 우위를 도모할 수 있습니다.
- **하만카돈**은 음악 감상에 초점을 맞춘 균형 잡힌 사운드 프로필로, 부드러운 고음과 풍부한 중저음을 제공하여 클래식, 재즈 등 다양한 음악 장르에서 높은 음질을 경험할 수 있습니다. 하이파이 음향 시스템으로서 고해상도 오디오를 지원합니다.

### 2. 디자인 및 사용성
- **JBL 제품**은 젊고 역동적인 감각의 디자인으로, 게이밍 헤드셋은 RGB 조명과 커스터마이징 기능이 탑재되어 있어 사용자 개성 표현이 가능합니다. 휴대성과 내구성이 뛰어나고 다양한 무선 연결 방식을 지원합니다.
- **하만카돈 제품**은 고급스러운 메탈과 우드 소재를 사용해 세련되고 미니멀한 디자인을 추구하며, 인테리어와 조화를 이루는 프리미엄 가전으로 평가받습니다. 주로 가정용 프리미엄 오디오 시스템에 적합합니다.

### 3. 가격대 및 시장 포지션
- **JBL**은 중저가~중가 제품군 중심으로 가격 접근성이 좋아 게이머, 젊은 층, 캐주얼 음악 청취자에게 적합합니다.
- **하만카돈**은 중고가 이상의 프리미엄 라인업으로, 고품질 음향을 중요시하는 하이엔드 사용자와 오디오 애호가 대상입니다.

### 4. 기술적 차이
- **JBL**은 최신 게이밍 오디오 기술과 헤드 트래킹 기능, 커스터마이징 소프트웨어가 강점입니다. 예를 들어, JBL QuantumSPHERE 360은 사용자의 머리 움직임을 감지해 360도 공간 음향을 정확히 유지해주어 게임 내 위치감 향상에 도움을 줍니다.
- **하만카돈**은 DSP와 하이파이 오디오 기술에 집중하며, 무선 멀티룸 기능과 스마트 홈 연동 기능을 갖춘 제품들이 많아 고급 오디오 환경 구축에 적합합니다.

---

# 요약
| 항목       | JBL                          | 하만카돈 (Harman Kardon)            |
|------------|------------------------------|------------------------------------|
| 음질       | 강력한 베이스, 게이밍에 최적화 | 균형 잡힌 고해상도 하이파이 음질    |
| 디자인     | 젊고 역동적, RGB 조명 포함    | 고급스러움, 미니멀하고 세련됨       |
| 가격대     | 중저가~중가형                  | 중고가~고가 프리미엄                 |
| 주요 기술  | 공간 음향, 헤드 트래킹, EQ 조절 | DSP, 무선 멀티룸, 스마트 연동      |

---

필요한 제품 추천이나 자세한 기능 설명이 필요하시면 언제든 문의 주세요! 삼성전자를 통해 공식 유통되는 JBL과 하만카돈 제품을 통해 본인 취향과 사용 목적에 맞는 최적의 오디오 경험을 즐기시기 바랍니다. 😊

---

*참고: 최신 제품 정보와 가격, 프로모션은 삼성전자 공식 홈페이지 및 인증 판매처에서 확인하시는 것이 가장 정확합니다.*

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데, 새 이어버드를 기존 갤럭시 버즈와 함께 페어링하는 방법을 자세히 알려주세요. 한쪽 이어버드만 새로 구매했을 때 페어링 절차와 주의할 점을 알고 싶습니다.
Google Search Results: 10


안녕하세요! 갤럭시 버즈 이어버드 한쪽을 새로 구매했을 때 기존 이어버드와 함께 페어링(커플링)하는 방법과 주의사항을 자세히 안내해 드리겠습니다.

---

## 갤럭시 버즈 이어버드 한쪽 새로 구매 후 페어링(커플링) 방법

### 1. 기본 준비 사항
- 새로 구매한 이어버드와 기존 사용 중인 이어버드를 충전 케이스 좌우 슬롯에 정확히 맞게 넣습니다.
- 이어버드가 완전히 충전되어 있어야 하며, 충전 케이스 LED 상태를 확인합니다.

### 2. 이어버드 커플링 절차

#### (1) 이어버드 한 쪽씩 충전 케이스에 넣고 정상 안착 확인
- 충전 케이스를 열고 이어버드를 좌우 슬롯 홈에 맞게 넣습니다.
- 충전 케이스 LED가 녹색 또는 빨간색으로 깜빡이는지 확인하세요.
- 만약 LED가 빨간색으로 계속 깜빡인다면, 이어버드와 케이스 단자 접점에 이물질이나 부식이 없는지 점검하고, 충전 케이스를 닫았다 열었다 2회 반복한 후 다시 시도합니다.

#### (2) 양쪽 이어버드를 동시에 7초 이상 길게 터치
- 충전 케이스가 열린 상태에서, 이어버드의 터치 패드를 양쪽 모두 동시에 7초 이상 누르고 계세요.
- 이때 이어버드 배터리 상태 표시등이 녹색으로 3~4회 깜빡인 후 꺼지면 커플링이 완료됩니다.
- 만약 빨간색으로 계속 깜빡인다면, 다시 이어버드를 충전 케이스에서 분리했다가 넣은 후 같은 절차를 반복합니다.

#### (3) 모바일 기기에서 Galaxy Wearable 앱 실행 후 연결
- 블루투스를 켜고, Galaxy Wearable(갤럭시 웨어러블) 앱을 실행합니다.
- 새로 커플링된 이어버드를 선택하여 정상적으로 페어링합니다.
- 페어링 직후 양쪽 이어버드의 소프트웨어 버전이 다를 경우, 앱에서 업데이트 안내 팝업이 뜹니다. 업데이트를 진행하세요.

---

## 주의할 점 및 참고사항

- **배터리 타입 확인:** 새로 구매한 이어버드와 기존 이어버드의 배터리 타입이 반드시 동일해야 커플링이 가능합니다. (예: 2019년 9월 이전 생산과 이후 생산 제품 간 배터리 타입 차이 문제)
- **소프트웨어 버전 차이:** 양쪽 이어버드의 소프트웨어 버전이 다르면 페어링 모드 진입이 불가합니다. 모바일 기기에서 업데이트를 꼭 진행해 주세요.
- **충전 케이스 상태 확인:** 충전 케이스 단자에 이물질, 부식, 또는 침수 여부가 있으면 정상 커플링이 어렵습니다. 깨끗한 상태로 유지해주세요.
- **커플링 실패 시 조치:** 
  - 충전 케이스의 뚜껑을 닫았다 열기를 2회 반복 후 다시 시도하세요.
  - Galaxy Wearable 앱에서 강제 커플링 시도를 해볼 수 있습니다.
  - 그래도 실패하면 삼성전자 서비스센터 방문을 권장합니다.

---

## 서비스센터 방문 절차

- 한쪽 이어버드 분실 후 새로 구입 시, 서비스센터에 방문하여 A/S 접수를 하시면 이어버드 구입과 함께 양쪽 이어버드 커플링 작업을 엔지니어가 진행해 줍니다.
- 직접 커플링이 어려운 모델도 있으니, 문제가 지속될 경우 꼭 서비스센터 방문을 추천드립니다.

---

필요 시 아래 절차를 참고하세요:

1. 양쪽 이어버드를 충전 케이스에 넣기
2. 양쪽 이어버드 터치패드 7초 이상 길게 누르기
3. 충전 케이스 LED 녹색 깜빡임 확인 후 꺼짐 → 커플링 완료
4. Galaxy Wearable 앱에서 페어링 및 소프트웨어 업데이트 진행

---

더 궁금한 점 있으시면 언제든지 문의해 주세요! 감사합니다.

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 S24 무게 차이
LLM Query: 삼성전자 모델 S25의 무게가 S24 모델과 비교했을 때 얼마나 차이가 나는지 알려주세요. 두 모델의 무게 차이와 관련된 구체적인 수치를 알고 싶습니다.
Google Search Results: 10
Request failed: [SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] ssl/tls alert handshake failure (_ssl.c:1010)


삼성전자 갤럭시 S25 모델과 갤럭시 S24 모델의 무게 차이는 다음과 같습니다.

| 모델           | 무게 (g)      | 비고                          |
|----------------|---------------|-------------------------------|
| 갤럭시 S25      | 162g          | 두께 7.2mm                   |
| 갤럭시 S24      | 168g          | 두께 7.6mm                   |
| **무게 차이**   | **6g 감소**   | S25가 S24보다 6g 가벼움       |

- 갤럭시 S25는 S24에 비해 약 6g 가벼워졌으며, 두께도 7.6mm에서 7.2mm로 0.4mm 더 얇아져 휴대성과 그립감이 개선되었습니다.
- 이러한 경량화는 소재 개선과 내부 부품 최적화를 통해 이루어졌으며, 사용자들이 장시간 사용 시 손목에 가해지는 부담을 줄여주는 효과가 있습니다.

요약하자면, 갤럭시 S25는 갤럭시 S24보다 무게가 약 6g 가벼워져서 더 편안한 사용감을 제공합니다.